今天学到pandas的一个扩展库-``pandas-profiling``，该库可以从dataframe中生成报告文件。实际上df.describe()就是能实现类似的功能，但是对于数据分析初期的数据探索略微有些鸡肋。

``pandas-profiling`` 可以在html动态交互页面中呈现如下统计信息：
- 基础信息：类型、唯一值、缺失值
- 分位数统计量：如最小值，Q1，中位数，Q3，最大值，范围，四分位数范围
- 描述性统计量：均值、标准差、和、中位绝对偏差、变异系数、峰度、偏度
- 最常出现的值
- 直方图
- 相关系数：高度相关的变量、斯皮尔曼和皮尔森矩阵

### 安装

In [1]:
!pip3 install pandas_profiling

  Stored in directory: /Users/suosuo/Library/Caches/pip/wheels/bf/c5/d0/7278ea92cf6195c9bef7d9b1b63767fc16dc490a2402287067
Successfully built pandas-profiling
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### pandas-profiling例子
这里我们使用**气象学会**的综合数据集，该数据集包含所有已知陨石着陆的信息。 由Javier de la Torre收集，提供了一个由34,513个陨石组成的XLS文件，其中包括以下字段：

```
location
type_of_meteorite
mass_g
fell_found
year、database
coordinate_1
coordinate_1
coordinates_2
cartodb_id
created_at
updated_at
year_date
longitude
latitude
geojson
```

我们先读取数据看看

In [31]:
import pandas as pd
import pandas_profiling
import numpy as np

df = pd.read_csv('Meteorite_Landings.csv', parse_dates=['year'])
df.head()

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation,States,Counties
0,Aachen,1,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)",NaN,NaN
1,Aarhus,2,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)",NaN,NaN
2,Abee,6,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)",NaN,NaN
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)",NaN,NaN
4,Achiras,370,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)",NaN,NaN


### 生成一些实验数据
为了方便学习pandas_profiling库，我们生成一些缺失值、随机值、高度自相关等特点的信息。

In [32]:
#pandas不支持1880年之前的日期，所以遇到errors我们需要设置处理方式。
#pandas中error有 raise、ignore和coerce三种。coerce是将不支持的日期设置为NaN
df['year'] = pd.to_datetime(df['year'], errors='coerce')

# 常量
df['source'] = "NASA"

#随机生成df.shape[0]个布尔值
df['boolean'] = np.random.choice([True, False], df.shape[0])

# 随机生成df.shape[0]个1或者A
df['mixed'] = np.random.choice([1, "A"], df.shape[0])

# 新建一个与reclat高度相关的reclat_city变量
df['reclat_city'] = df['reclat'] + np.random.normal(scale=5,size=(len(df)))

duplicates_to_add[u'name'] = pd.DataFrame(df.iloc[0:10][u'name']) + " copy"

df = df.append(duplicates_to_add, ignore_index=True)

df.tail(5)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Counties,GeoLocation,States,boolean,fall,id,mass (g),mixed,name,nametype,recclass,reclat,reclat_city,reclong,source,year
45721,NaN,"(32.1, 71.8)",NaN,NaN,Fell,379,4239.0,NaN,Adhi Kot copy,Valid,EH4,32.10000,NaN,71.80000,NaN,01/01/1919 12:00:00 AM
45722,NaN,"(44.83333, 95.16667)",NaN,NaN,Fell,390,910.0,NaN,Adzhi-Bogdo (stone) copy,Valid,LL3-6,44.83333,NaN,95.16667,NaN,01/01/1949 12:00:00 AM
45723,NaN,"(44.21667, 0.61667)",NaN,NaN,Fell,392,30000.0,NaN,Agen copy,Valid,H5,44.21667,NaN,0.61667,NaN,01/01/1814 12:00:00 AM
45724,NaN,"(-31.6, -65.23333)",NaN,NaN,Fell,398,1620.0,NaN,Aguada copy,Valid,L6,-31.60000,NaN,-65.23333,NaN,01/01/1930 12:00:00 AM
45725,NaN,"(-30.86667, -64.55)",NaN,NaN,Fell,417,1440.0,NaN,Aguila Blanca copy,Valid,L,-30.86667,NaN,-64.55000,NaN,01/01/1920 12:00:00 AM


### 输出分析报告
在notebook的Cell内输出dataframe的分析报告

In [48]:
pandas_profiling.ProfileReport(df)

![](img/01.gif)
具体我们看看详细的报告，注意图片中的Toggle Detail也是可以点击查看更详细的信息。
![](img/01.png)
![](img/02.png)
![](img/03.png)
![](img/04.png)
![](img/05.png)
![](img/06.png)

报告真的很详细美观，很适合用到数据探索阶段。
### 将报告输出到html文件中
有时候我们需要将分析报告分享给其他人，这时候我们就需要将报告输出为html文件方便其他人查看。

In [43]:
pfr = pandas_profiling.ProfileReport(df)
pfr.to_file("report.html")

![](img/02.gif)

### profiling对象行内显示
刚刚上面的pfr变量是profiling对象，可以直接在行内显示具体的报告信息

In [45]:
pfr

![](img/03.gif)

### 高级用法
pandas_profiling.ProfileReport()函数有以下参数，我们可以根据需要自定义
- bins (int):直方图的列数(默认为10).

**相关性设置:**
- check_correlation： 布尔型值，是否检验相关性，默认为True
- correlation_threshold: 浮点值（值域0-1），该值用来设置判定两个变量之间是否为相关。默认阈值为0.9，在社科类中，这个参数可以用来诊断变量间的共线性问题
- correlation_overrides:  (list): Variable names not to be rejected because they are correlated (None by default).
- check_recoded (boolean): 布尔型值，用来诊断记录间是否存在相关性，因为这种计算比较消耗计算能力，所以建议只在小样本量的情况下使用。
- pool_size:整数型，并行加快计算，该参数默认与CPU的个数相等。